#### Import dep

In [4]:
import os
import sys
import pandas as pd
import openai
import pandas as pd
from glob import glob
from tqdm import tqdm
import json
from pathlib import Path
from dotenv import load_dotenv


In [5]:
#machine = "local"
machine = "paperspace"

if machine == "local":
    src_dir= Path.cwd().parent    
elif machine == "paperspace":
    src_dir = Path("/notebooks/inferess-relation-extraction/")

sys.path.append(str(src_dir))


In [6]:

#load_dotenv()
#openai.api_key = os.getenv("OPENAI_API_KE")


# import annotation methods
from src.labels_generator import (generate_relations,relation_search,resort_relation, get_completion,
                                  deserialize_relations,
                                  generate_relations_with_explanation,
                                  relations_tupled,
                                 create_sorted_relation)

# Load matcher
from src.matcher.core import SimCSE_Matcher
matcher = SimCSE_Matcher(str(src_dir/ 'artifacts/matcher_model'))

main_relations = ["supplier", "customer"]
replaces = {"sentence": "{sentence}"}
# Replace the keys with values for unified relation direction
relations_map = {"customer": "supplier"}

### Filters for train data

#### LLM labelled data

In [7]:
data1 = pd.read_excel(str("/storage/test_pipeline_data/pipeline_train_data/llm_relations_all_label_1_v2_3.xlsx"))

# data2 = pd.read_excel(str("/storage/test_pipeline_data/pipeline_train_data/huge_train_llm_aligned_v2_3_0_1300.xlsx"))
# data3 = pd.read_excel(str("/storage/test_pipeline_data/pipeline_train_data/huge_train_complex_sents_llm_v2_3.xlsx"))
# data1 = data1[(data1["agreement_relation"] != "other") & (data1["license_relation"] != "other")]
# data2 = data2[(data2["agreement_relation"] != "other") & (data2["license_relation"] != "other")]
# data3 = data3[(data3["agreement_relation"] != "other") & (data3["license_relation"] != "other")]
# data1.to_excel( str(src_dir / "data/raw/llm_relations_all_label_1_v2_3.xlsx"), index=False)
# data2.to_excel( str(src_dir / "data/raw/huge_train_llm_aligned_v2_3_0_1300.xlsx"), index=False)
# data3.to_excel( str(src_dir / "data/raw/huge_train_complex_sents_llm_v2_3.xlsx"), index=False)

In [56]:
data1.columns

Index(['index', 'sentence', 'Label', 'org_groups', 'inf_relations', 'entity_1',
       'entity_2', 'sme_relations', 'explanation', 'relation_completion',
       'relations', 'align', 'concepts', 'concept explanation',
       'concept_class', 'agreement_relation', 'license_relation',
       'supply_chain_relation'],
      dtype='object')

In [40]:
data1[["sme_relations", "relations", "inf_relations" ]].head()

,sme_relations,relations,inf_relations
0,"['IMMUNOGEN INC', 'supplier', 'Roche']","[['IMMUNOGEN Inc', 'supplier', 'Roche']]",customer
1,"['Zhejiang Yingte Pharmaceutical Co. Ltd', 'su...","[['HuaDong Pharmaceutical Co, Ltd', 'supplier'...",supplier
2,"['WPX', 'supplier', 'NGL Energy Partners']","[['WPX', 'supplier', 'NGL Energy Partners']]",supplier
3,"['Xilinx Inc ’s', 'supplier', 'Avnet Inc']","[['Xilinx Inc', 'supplier', 'Avnet Inc']]",customer
4,"['VIAVI SOLUTIONS INC.', 'supplier', 'SICPA']","[['VIAVI SOLUTIONS Inc', 'supplier', 'SICPA']]",customer


#### Positive data

In [74]:
required_columns = ['accession_number', 'index', 'sentence', 'Label', 'org_groups', 'inf_relations', 
                    'entity_1', 'entity_2', 'sme_relations',  'relations', 
                    'concept_class']

In [45]:
weakly_labeled_data_pos = pd.read_excel("./test_pipeline_data/huge_set_deduped_data/huge_train_dedup_sc_cc_re.xlsx")

In [46]:
weakly_labeled_data_pos.columns

Index(['level_0', 'index', 'accessionNumber', 'filer', 'firstEntity',
       'relationship', 'secondEntity', 'sentence', 'sme_relations',
       'duplicate_sentences', 'sent_size', 'clause_size', 'deduped',
       'sc_score', 'sc_label', 'cc_class', 'cc_class_2', 'cc_class_score',
       'cc_class_2_score', 'spans', 'org_groups', 'aliases', 'num_orgs',
       'filtered_org_groups', 'num_orgs_filter', 're_relations', 're_score',
       're_rel_label', 're_pred_match', 'data_status', 'matching_sent_count'],
      dtype='object')

In [42]:
weakly_labeled_data_pos[["sme_relations", "relationship" ]].head()

,sme_relations,relationship
0,"['ADVANCED MICRO DEVICES INC', 'supplier', 'MI...",customer
1,"['ADVANCED MICRO DEVICES INC', 'supplier', 'MI...",customer
2,"['ADVANCED MICRO DEVICES INC', 'supplier', 'MI...",customer
3,"['ADVANCED MICRO DEVICES INC', 'supplier', 'MI...",customer
4,"['GLOBALFOUNDRIES Inc', 'supplier', 'AMD']",supplier


In [47]:
# sc model was not confident on these sentences, but re models scored them high with correct relation, 
# sentence is matching at least 1 training row in existing training data

# create a copy of weakly_labeled_data_pos with following conditions


weakly_labeled_data_pos_1 = weakly_labeled_data_pos[(weakly_labeled_data_pos["sc_label"] == 1) &
                                                    (weakly_labeled_data_pos["sc_score"] < 0.95 ) &
                                                    (weakly_labeled_data_pos["re_rel_label"].isin(["supplier", "customer"])) &
                                                    (weakly_labeled_data_pos["re_score"] > 0.95) &
                                                    (weakly_labeled_data_pos["re_pred_match"] == True) &                                                     
                                                    (weakly_labeled_data_pos["matching_sent_count"] > 0)].copy()

                                                    
weakly_labeled_data_pos_1.shape


(3039, 31)

In [99]:
# Apply eval on column sme_relations
weakly_labeled_data_pos_1.loc[:, "sme_relations"] = weakly_labeled_data_pos_1["sme_relations"].apply(eval)


# rename columns
weakly_labeled_data_pos_1.rename(columns={"accessionNumber": "accession_number",
                                         "relationship": "inf_relations", 
                                          "firstEntity": "entity_1",
                                          "secondEntity": "entity_2"},
                                 inplace=True)
## Dervie columns   
# Label
# relations
# concept_class
weakly_labeled_data_pos_1.loc[:, "Label"] = 0
weakly_labeled_data_pos_1.loc[:, "relations"] = [[sme_relation] for sme_relation in weakly_labeled_data_pos_1["sme_relations"].to_list()]
weakly_labeled_data_pos_1.loc[:, "concept_class"] = weakly_labeled_data_pos_1["cc_class"]

In [100]:
weakly_labeled_data_pos_1[required_columns].to_excel( str(src_dir / "data/raw/huge_train_pipe_labelled_pos_more_to_sc.xlsx"), index=False)

#### Negative data

In [103]:
weakly_labeled_data_neg = pd.read_excel("./test_pipeline_data/huge_set_deduped_data/huge_neg_train_dedup_sc_cc_re.xlsx")

In [70]:
weakly_labeled_data_neg.columns

Index(['Acc no', 'filer', 'secondEntity', 'sentence', 'Prediction', 'Score',
       'relationship', 'index', 'sme_relations', 'sent_size', 'clause_size',
       'deduped', 'duplicate_sentences', 'spans', 'org_groups', 'aliases',
       'num_orgs', 'filtered_org_groups', 'num_orgs_filter', 'sc_score',
       'sc_label', 'cc_class', 'cc_class_2', 'cc_class_score',
       'cc_class_2_score', 're_relations', 're_score', 're_rel_label',
       're_pred_match'],
      dtype='object')

In [104]:
weakly_labeled_data_neg[["sme_relations", "relationship" ]].head()

,sme_relations,relationship
0,"['EMA Financial', 'other', '12 Retech Corp']",other
1,"['EMA Financial', 'other', '12 Retech Corp']",other
2,"['SBI Investments', 'other', '12 Retech Corp']",other
3,"['SBI Investments', 'other', '12 Retech Corp']",other
4,"['SBI Investments', 'other', '12 Retech Corp']",other


In [105]:
# sc label is 1, but re_model corrected it to `other`, we removed re_score filter to get more data 
weakly_labeled_data_neg_1 = weakly_labeled_data_neg[ (weakly_labeled_data_neg["sc_label"] == 1) &
                                                    (weakly_labeled_data_neg["re_rel_label"].isin(["other"])) &
                                                    #(weakly_labeled_data_neg["re_score"] > 0.95) &
                                                    (weakly_labeled_data_neg["re_pred_match"] == True)                                                      
                                                     #(weakly_labeled_data_neg["matching_sent_count"] > 0) 
                                                     ].copy()

weakly_labeled_data_neg_1.shape


(1137, 29)

In [108]:
# Apply eval on column sme_relations
weakly_labeled_data_neg_1.loc[:, "sme_relations"] = weakly_labeled_data_neg_1["sme_relations"].apply(eval)

# rename columns
weakly_labeled_data_neg_1.rename(columns={"Acc no": "accession_number",
                                         "relationship": "inf_relations", 
                                          "filer": "entity_1",
                                          "secondEntity": "entity_2"},
                                 inplace=True)
## Dervie columns   
# Label
# relations
# concept_class
weakly_labeled_data_neg_1.loc[:, "Label"] = 0
weakly_labeled_data_neg_1.loc[:, "relations"] = [[sme_relation] for sme_relation in weakly_labeled_data_neg_1["sme_relations"].to_list()]
weakly_labeled_data_neg_1.loc[:, "concept_class"] = weakly_labeled_data_neg_1["cc_class"]



In [73]:
weakly_labeled_data_neg_1.columns

Index(['accession_number', 'entity_1', 'entity_2', 'sentence', 'Prediction',
       'Score', 'inf_relations', 'index', 'sme_relations', 'sent_size',
       'clause_size', 'deduped', 'duplicate_sentences', 'spans', 'org_groups',
       'aliases', 'num_orgs', 'filtered_org_groups', 'num_orgs_filter',
       'sc_score', 'sc_label', 'cc_class', 'cc_class_2', 'cc_class_score',
       'cc_class_2_score', 're_relations', 're_score', 're_rel_label',
       're_pred_match', 'Label', 'relations', 'concept_class'],
      dtype='object')

In [109]:
weakly_labeled_data_neg_1[required_columns].to_excel( str(src_dir / "data/raw/huge_train_pipe_labelled_neg_more_to_sc.xlsx"), index=False)


### Transform LLM Data into RE Data

#### Load NER Tagger

In [21]:
from src.language_model.spacy_loader import SpacyLoader
spacy_loader = SpacyLoader("en_core_web_trf",
                           entity_matcher= str(src_dir / "artifacts/matcher_model/"),
                          load_matcher=True)

2023-10-25 07:48:49,728 — 🌌 spaCy — INFO — Language model used is en_core_web_trf
2023-10-25 07:48:49,729 — 🌌 spaCy — INFO — spaCy Work On GPU


#### Load project configuration

In [7]:
# Read project configuration
import yaml
from src.utils.data import dict2dot


with open(src_dir/ 'params.yaml' ,'r') as o:
    config = dict2dot(yaml.safe_load(o))


#### Helper Functions

In [8]:

def eval_relations(relation, default=[]):
    try:
        re = eval(relation)
    except:
        re = default
    return re

def eval_relation_data(output, relations_map):
    # Resort the sme_relations to unify the relations directions
    if not isinstance(output['sme_relations'].iloc[0], list):
        tqdm.pandas(desc="Eval sme_relations")
        output['sme_relations'] = output['sme_relations'].progress_apply(eval)
        
        
    if 'relations' in output.columns:
        if not isinstance(output['relations'].iloc[0], list):
            tqdm.pandas(desc="Eval relations")
            output['relations'] = output['relations'].progress_apply(eval_relations)
    else:
        output['relations'] = None
    if not isinstance(output['org_groups'].iloc[0], dict):
        tqdm.pandas(desc="Eval org_groups")
        output['org_groups'] = output['org_groups'].progress_apply(eval_relations, default={})  
    tqdm.pandas(desc="Resort sme relations")
    output['sme_relations'] = output['sme_relations'].progress_apply(lambda x:\
                              resort_relation((x[0], x[1], x[2]),
                                            relations_map))
def process_labeled_data(data,
                         text_col:str,
                         relation_tuple:str,
                        relations_map:dict): 
    # Create org_groups if not exist
    if not all([x in data.columns for x in ['spans', 'org_groups']]):
        from src.language_model.spacy_loader import SpacyLoader
        spacy_loader = SpacyLoader("en_core_web_trf",
                                   entity_matcher=str(src_dir/ 'artifacts/matcher_model'),
                                  load_matcher=True)
        sents, spans, org_groups, aliases = spacy_loader.predictor(data[text_col])
        data[text_col]= sents
        data.loc[:, text_col] = sents
        data.loc[:, "spans"] = spans
        data.loc[:, "org_groups"] = org_groups
    # deserialize data
    eval_relation_data(data, relations_map)
    



In [9]:

import re
import json
import time
from typing import Tuple, List, Text, Dict
from collections import defaultdict
from tqdm import tqdm
import traceback
from typing import Text 
from itertools import chain
from copy import copy
import pandas as pd
import numpy as np
import random

from operator import is_not
from functools import partial
from src.labels_generator.utils import check_relation_tuples


def get_other_relations(org_groups, max_others:int):
    """
    Returns a list of other relations between companies based on the dictionary of company groups passed as input.
    The maximum number of other relations is determined by the max_others parameter.

    @params
    -------
    org_groups (dict): A dictionary with company names as keys and groups as values.
    max_others (int): The maximum number of other relations to return.

    @returns
    --------
    list: A list of other relations between companies.
    """
    # Collect all unique company names from the org_groups dictionary
    orgs = list(org_groups.keys())
    orgs = {k:None for k in set(orgs)}

    # Create a dictionary to map each unique company name to an ID
    orgs_ids = {k:i for i,k in enumerate(set(orgs))}
    ids2org = {i:k for i,k in enumerate(set(orgs_ids))}

    # Create a list of all possible pair-wise combinations of the values in 'ids2org', and randomly choose max_others of those combinations
    available_relations = []
    comp_keys = list(ids2org.keys())
    for i in range(len(comp_keys)):
        for j in range(i+1, len(comp_keys)):
            relation_t = tuple(sorted([comp_keys[i], comp_keys[j]]))
            available_relations.append(relation_t)

    other_ids = set(available_relations)

    other_relations = []
    if len(other_ids) > max_others:
        other_ids = random.sample(list(other_ids), max_others)

    # Map the IDs back to the company names and create a list of other relations
    for pair in other_ids:
        c1 = ids2org[pair[0]]
        c2 = ids2org[pair[1]]
        other_relations.append((c1,'other', c2))

    return other_relations

def extract_relations_from_llm(datapoint,
                                matcher,
                                text_col:str="sentence",
                                relations_key:str='relations',
                                threshold:float=0.9,
                               max_others=3):
    
    """
    Create a dataset for relation extraction training.
    @params
    -------
    - datapoint: A dictionary containing the following keys:
        - org_groups: A dictionary of company names associated with an integer identifier.
        - relations: A list of tuples representing the relationship between companies.
    - matcher: An instance of the `StringMatcher` class.
    - text_col: The key in the datapoint dictionary where the text to match can be found.
    - relations_key: The key in the datapoint dictionary where the relations can be found.
    - threshold: The similarity threshold for matching company names.

    @returns
    --------
    - llms_relations: A list of tuples representing the relationships between companies that were successfully matched.
    - other_relations: A list of tuples representing the relationships between companies that were not matched.

    @raises
    -------
    - ValueError: If the relations list in the datapoint is invalid.
    """
    r_others = True
    # establish org_groups
    group2id = datapoint['org_groups']
    id2group = defaultdict(list)
    for k,v in group2id.items():
        id2group[v].append(k)
    
    # define llms relations
    relations = datapoint[relations_key]
    
    # build index for org_groups
    if len(group2id) > 0:
        matcher.build_index(list(group2id.keys()))
    
    # Assert the relations on the right format
    if not check_relation_tuples:
        raise ValueError("Invlid relations list on the datapoint, must be List[Tuple[Text, Text, Text]]")
    # Collect all companies mentioned in the relations and create a dictionary with each unique company as a key
    llms_companies = []
    if isinstance(relations, list):
        for relation in relations:
            llms_companies += [relation[0] , relation[2]]
            
            
    # match the llm_companies to assign id according to group2id
    llms_co_matches = matcher.search(llms_companies, threshold=threshold, top_k=2)\
                      if len(llms_companies) > 0 else []
    llms_ids = {}
    for i,co_match in enumerate(llms_co_matches):
        if len(co_match) > 0:
            llms_ids[llms_companies[i]] = group2id[co_match[0][0]]
        elif llms_companies[i] in datapoint[text_col]:
            group2id[llms_companies[i]] = len(group2id)+1
            id2group[len(group2id)+1] = [llms_companies[i]]
            llms_ids[llms_companies[i]] = len(group2id)+1
    
    # llms_ids = {llms_companies[i]: group2id.get(x[0][0], None) \
    #             if len(x) > 0 else None for i,x in enumerate(llms_co_matches)}
    
    
    llms_names = {k:id2group[v][0] if v is not None else None for k,v in llms_ids.items() }
    # in case the lmm companies not all aligned with NER org_groups, don't involve other
    if None in llms_ids.values():
        r_others = False

    ids_set = list(set(filter(partial(is_not, None), llms_ids.values())))
        
    availabel_relations = []
    for i in range(len(ids_set)):
        for j in range(i+1, len(ids_set)):
            relation_t = tuple(sorted([ids_set[i], ids_set[j]]))
            availabel_relations.append(relation_t)
    exist_relations = []
    llms_relations = []
    if isinstance(relations, list):
        for relation in relations:
            c1, c1_name = relation[0], llms_names.get(relation[0])
            c2, c2_name = relation[2], llms_names.get(relation[2])
            relation = relation[1]
            if None in  [c1, c2 , c1_name, c2_name]:
                continue

            llms_relations.append((c1_name, relation, c2_name))

            c1_id = llms_ids.get(c1)
            c2_id = llms_ids.get(c2)    
            exist_relations.append(tuple(sorted([c1_id, c2_id])))
    
    other_ids = set(availabel_relations) ^ set(exist_relations)
    # if len(other_ids)>max_others:
    #     other_ids = random.sample(other_ids, max_others)
    other_relations = []
    if r_others:
        for pair in list(other_ids): 
            c1 = id2group[pair[0]][0]
            c2 = id2group[pair[1]][0]
            other_relations.append((c1,'other', c2))
        
    if len(llms_relations) == 0 and len(other_relations) == 0:
        other_relations = get_other_relations(group2id, max_others)
        
    return llms_relations, other_relations


def create_re_dataset(
                      output,
                      matcher,
                      text_col:str="sentence",
                      relations_key:str='relations',
                      threshold:float=0.9,
                      max_others:int=3,
                      basic_columns:list=[],
                      ) -> pd.DataFrame:
    """
    Create a relation extraction dataset.

    @params
    -------
    - output: A pandas dataframe containing the following keys:
        * sentence: A string representing a sentence containing relevant text.
        * relations: A list of tuples representing the relationship between companies.
    - matcher: An instance of the `StringMatcher` class.
    - text_col: The key in the output dataframe where the text to match can be found.
    - relations_key: The key in the output dataframe where the relations can be found.
    - threshold: The similarity threshold for matching company names.

    @returns
    --------
    - dataset: A pandas dataframe containing the following columns:
    - idx: A unique identifier for the datapoint.
    - sentence: A string representing the sentence containing relevant text.
    - entity_2: A string representing the second entity in the relation.
    - relation: A string representing the relation between the two entities.
    - entity_1: A string representing the first entity in the relation.
    """

    # Apply the `extract_relations_from_llm` function to each datapoint in the output dataframe
    results = output.apply(lambda x: extract_relations_from_llm(datapoint=x,
                            matcher=matcher,
                            text_col=text_col,
                            relations_key=relations_key,
                            threshold=threshold), axis=1)

    # Create new columns in the output dataframe to store the results of the `extract_relations_from_llm` function
    output['llm_relations']  = results.apply(lambda x : x[0])
    output['other_relations']  = results.apply(lambda x : x[1])
    relation_columns = ['llm_relations', 'other_relations']
    # Create a list of all possible pair-wise combinations of the values in 'ids2org', and randomly choose 5 of those combinations
    columns = relation_columns + [text_col, relations_key] + basic_columns
    re_dataset = []
    for i, row in output[columns].iterrows():
        row = row.to_dict()
        for r_column in relation_columns:
            # Iterate over relations and ingest row for each relation
            for relation_tuple in row[r_column]:
                row['entity_2'] = relation_tuple[0]
                row['relation'] = relation_tuple[1]
                row['entity_1'] = relation_tuple[2]
                re_dataset.append(dict(row))

    # Return a pandas dataframe containing the relation extraction dataset
    dataset = pd.DataFrame(re_dataset)[[ text_col,
                                        'entity_2',
                                        'relation',
                                        'entity_1'] + basic_columns]
    return dataset
# method to add Label based on the relations columns
def sc_label_from_relations(relation_tuples, main_relations):
    if not relation_tuples:
        return 0
    if len(relation_tuples) == 0:
        return 0
    for relation_tuple in relation_tuples:
        if len(relation_tuple) != 3:
            continue
        elif  relation_tuple[1] in main_relations:
            return 1
    return 0


#### Define and read data files

In [12]:
import yaml
# define configuration file of the files
#dataset_name = "llm_aligned_0_1_huge_complex"

dataset_name = "llm_aligned_0_1_huge_complex"

with open(src_dir/ 'data/config/{}.yaml'.format(dataset_name)) as o:
    data_files = yaml.safe_load(o)

In [13]:
data_files

{'all_0': {'dir': '/notebooks/inferess-relation-extraction/data/raw/verified_train_files/llm_relations_other_from_label_0_v2_3.xlsx'},
 'all_1': {'dir': '/notebooks/inferess-relation-extraction/data/raw/verified_train_files/llm_relations_all_label_1_v2_3.xlsx'},
 'all_other': {'dir': '/notebooks/inferess-relation-extraction/data/raw/verified_train_files/llm_relations_other_relation_v2_3.xlsx'},
 'huge_1': {'dir': '/notebooks/inferess-relation-extraction/data/raw/verified_train_files/huge_train_llm_aligned_v2_3_0_1300.xlsx'},
 'huge_1_complex': {'dir': '/notebooks/inferess-relation-extraction/data/raw/verified_train_files/huge_train_complex_sents_llm_v2_3.xlsx'}}

In [14]:

# assert all the keys contain directory
assert all([x.get('dir') for x in data_files.values()])

#### Prepare data to be in `RE_FORM`

In [15]:
# read file and store each into it's coresponding key_value
for key in list(data_files.keys()):
    print(key,'\n--------------')
    data_files[key]['data'] = pd.read_excel(data_files[key]['dir'])
    process_labeled_data(data_files[key]['data'], "sentence", "relations", relations_map)
    data_files[key]['data']['idx'] = ["{}_{}".format(key, i)\
                                      for i in range(data_files[key]['data'].shape[0])]
    data_files[key]['data']['Label'] = data_files[key]['data']['relations']\
                                        .apply(lambda x : sc_label_from_relations(x, main_relations))

all_0 
--------------
2023-10-25 07:46:39,316 — 🌌 spaCy — INFO — Language model used is en_core_web_trf
2023-10-25 07:46:39,317 — 🌌 spaCy — INFO — spaCy Work On GPU


Resort sme relations: 100%|██████████| 877/877 [00:00<00:00, 499647.46it/s]


all_1 
--------------
2023-10-25 07:47:01,600 — 🌌 spaCy — INFO — Language model used is en_core_web_trf
2023-10-25 07:47:01,601 — 🌌 spaCy — INFO — spaCy Work On GPU


Resort sme relations: 100%|██████████| 1767/1767 [00:00<00:00, 330700.78it/s]

all_other 
--------------
2023-10-25 07:47:13,609 — 🌌 spaCy — INFO — Language model used is en_core_web_trf
2023-10-25 07:47:13,610 — 🌌 spaCy — INFO — spaCy Work On GPU



Resort sme relations: 100%|██████████| 111/111 [00:00<00:00, 280631.55it/s]

huge_1 
--------------
2023-10-25 07:47:17,562 — 🌌 spaCy — INFO — Language model used is en_core_web_trf
2023-10-25 07:47:17,563 — 🌌 spaCy — INFO — spaCy Work On GPU



Resort sme relations: 100%|██████████| 688/688 [00:00<00:00, 492773.42it/s]


huge_1_complex 
--------------
2023-10-25 07:47:24,253 — 🌌 spaCy — INFO — Language model used is en_core_web_trf
2023-10-25 07:47:24,254 — 🌌 spaCy — INFO — spaCy Work On GPU


Resort sme relations: 100%|██████████| 1550/1550 [00:00<00:00, 493784.84it/s]


In [16]:
data_files.keys()

dict_keys(['all_0', 'all_1', 'all_other', 'huge_1', 'huge_1_complex'])

In [17]:
for key in list(data_files.keys()):
    print("Columns in {}: {}".format(key,data_files[key]['data'].columns))    
    print("-"*60)

Columns in all_0: Index(['index', 'sentence', 'Label', 'org_groups', 'inf_relations', 'entity_1',
       'entity_2', 'sme_relations', 'explanation', 'relation_completion',
       'relations', 'align', 'rd_conflict', 'concepts', 'concept explanation',
       'concept_class', 'spans', 'idx'],
      dtype='object')
------------------------------------------------------------
Columns in all_1: Index(['index', 'sentence', 'Label', 'org_groups', 'inf_relations', 'entity_1',
       'entity_2', 'sme_relations', 'explanation', 'relation_completion',
       'relations', 'align', 'concepts', 'concept explanation',
       'concept_class', 'agreement_relation', 'license_relation',
       'supply_chain_relation', 'spans', 'idx'],
      dtype='object')
------------------------------------------------------------
Columns in all_other: Index(['index', 'sentence', 'Label', 'org_groups', 'inf_relations', 'entity_1',
       'entity_2', 'sme_relations', 'explanation', 'relation_completion',
       'relatio

#### Create `RE_FORM` dataset for each file

In [18]:
config.base.basic_columns

['Label', 'concept_class', 'idx', 'org_groups', 'spans']

In [22]:
# read file and store each into it's coresponding key_value
for key in list(data_files.keys()):
    print(key,'\n--------------' )
    dataset = create_re_dataset(output=data_files[key]['data'],
                           matcher=spacy_loader.entity_matcher,
                           text_col="sentence",
                           relations_key="relations",
                           threshold=0.9,
                           max_others=1,
                           basic_columns=config.base.basic_columns)
    data_files[key]['dataset'] = dataset

10/25/2023 07:48:58 - INFO - faiss.loader -   Loading faiss with AVX2 support.
10/25/2023 07:48:58 - INFO - faiss.loader -   Successfully loaded faiss with AVX2 support.


all_0 
--------------
all_1 
--------------
all_other 
--------------
huge_1 
--------------
huge_1_complex 
--------------


In [23]:
all_dataset= pd.concat([data_files[k]['dataset'] for k in data_files.keys()], axis=0)

In [28]:
all_dataset.relation.value_counts()

other              6824
supplier           6619
financial_trade    1225
nothing             145
Name: relation, dtype: int64

,sentence,entity_2,relation,entity_1,Label,concept_class,idx,org_groups,spans
0,"For the fiscal year ended March 31, 2017, two ...","HuaDong Pharmaceutical Co, Ltd",supplier,CHINA JO JO DRUGSTORES Inc,1,revenue,all_1_0,"{'Zhejiang Yingte Pharmaceutical Co Ltd': 0, '...","[{'text': 'the fiscal year ended March 31, 201..."
1,"For the fiscal year ended March 31, 2017, two ...",Zhejiang Yingte Pharmaceutical Co Ltd,supplier,CHINA JO JO DRUGSTORES Inc,1,revenue,all_1_0,"{'Zhejiang Yingte Pharmaceutical Co Ltd': 0, '...","[{'text': 'the fiscal year ended March 31, 201..."
3,Since NGL Energy Partners December 2013 acquis...,WPX,supplier,NGL Energy Partners,1,investment_related,all_1_1,"{'NGL Energy Partners': 0, 'Gavilon Energy': 1...","[{'text': 'NGL Energy Partners', 'label': 'ORG..."
4,"Avnet Inc, one of Xilinx Inc ’s distributors, ...",Xilinx Inc ’s,supplier,Avnet Inc,1,supply_chain,all_1_2,"{'Xilinx Inc ’s': 0, 'Xilinx Inc’s': 0, 'Avnet...","[{'text': 'Avnet Inc', 'label': 'ORG', 'start'..."
5,"Under a license and supply agreement, VIAVI SO...",VIAVI SOLUTIONS Inc,supplier,SICPA,1,licensing_and_ip,all_1_3,"{'VIAVI SOLUTIONS Inc': 0, 'SICPA': 2}","[{'text': 'VIAVI SOLUTIONS Inc', 'label': 'ORG..."
...,...,...,...,...,...,...,...,...,...
4031,QUALCOMM INC/DE reports that QTL accounts rece...,QUALCOMM INC/DE,supplier,Huawei,1,revenue,huge_1_complex_1547,"{'QUALCOMM INC/DE': 0, 'Huawei': 1, 'vivo': 2,...","[{'text': 'QUALCOMM INC/DE', 'label': 'ORG', '..."
4032,QUALCOMM INC/DE reports that QTL accounts rece...,QUALCOMM INC/DE,supplier,Oppo,1,revenue,huge_1_complex_1547,"{'QUALCOMM INC/DE': 0, 'Huawei': 1, 'vivo': 2,...","[{'text': 'QUALCOMM INC/DE', 'label': 'ORG', '..."
4033,QUALCOMM INC/DE reports that QTL accounts rece...,QUALCOMM INC/DE,supplier,vivo,1,revenue,huge_1_complex_1547,"{'QUALCOMM INC/DE': 0, 'Huawei': 1, 'vivo': 2,...","[{'text': 'QUALCOMM INC/DE', 'label': 'ORG', '..."
4037,Certain of QUALCOMM Inc largest integrated cir...,QUALCOMM Inc,supplier,Samsung,1,unknown,huge_1_complex_1548,"{'QUALCOMM Inc': 0, 'Samsung': 1}","[{'text': 'QUALCOMM Inc', 'label': 'ORG', 'sta..."


In [18]:
all_dataset['idx'].unique().shape

(4871,)

#### Dataset for concept classifier

In [34]:
concept_class_remapping = {'agreement': ['supply_purchase_agreement',
               'services agreement',
               'agreement_and_partnership'],
'licensing_and_ip': ['royalties',
                     'licensing_and_ip',
                     'legal_and_regulatory'],
'supply_chain': ['supply_chain', 'product_related'],
'revenue': ['revenue', 'royalties'],
'real_estate': ['real_estate'],
'financial_statements': ['investment_related', 'financial_statements'],
'other': ['unknown'] 
}

# reverse the mapping
reverse_concept_class_remapping = {}
for k,v in concept_class_remapping.items():
    for x in v:
        reverse_concept_class_remapping[x] = k  
        

In [35]:
# remap the concept class
all_dataset["concept_class_remapped"] = all_dataset["concept_class"].apply(lambda x: reverse_concept_class_remapping.get(x, x))

In [36]:
# don't include other class
all_dataset = all_dataset[all_dataset["concept_class_remapped"] != "other"]


In [37]:
all_dataset["concept_class_remapped"].value_counts()

revenue                 6532
supply_chain            2590
agreement               1863
licensing_and_ip        1201
financial_statements    1152
real_estate              256
Name: concept_class_remapped, dtype: int64

#### Check dataset health

In [38]:
from src.utils.preprocess import word_search, Intersection


In [39]:
founded1 = all_dataset.apply(lambda x : word_search(x['entity_1'], x['sentence']), axis =1 )
founded2 = all_dataset.apply(lambda x : word_search(x['entity_2'], x['sentence']), axis =1 )

In [40]:
pd.set_option("display.max_colwidth", None)
all_dataset[founded2.apply(len) == 0]

,sentence,entity_2,relation,entity_1,Label,concept_class,idx,org_groups,spans,concept_class_remapped


In [41]:
(founded1.apply(len) == 0).sum()

0

In [42]:
(founded2.apply(len) == 0).sum()

0

In [43]:
(founded1.apply(len) == 0).sum()==0,(founded2.apply(len) == 0).sum()==0

(True, True)

In [44]:
all_dataset['relation'].value_counts()

other              6204
supplier           6171
financial_trade    1075
nothing             144
Name: relation, dtype: int64

#### Save dataset

In [45]:
# # load previous dataset
# previous_dataset = pd.read_json(str(src_dir / "data/raw/llm_aligned_0_1_huge_complex.json"))

# previous_dataset = previous_dataset[['sentence', 'entity_2', 'relation', 'entity_1', 'Label', 
#                                      'concept_class', 'idx', 'org_groups', 'spans']]


# # concat the previous dataset with the new one
# combined_all_dataset = pd.concat([all_dataset, previous_dataset], axis=0)


# combined_all_dataset.reset_index(drop=True).to_json(src_dir / 'data/raw/llm_aligned_0_1_huge_complex.json')

In [46]:
all_dataset.reset_index(drop=True).to_json(src_dir / 'data/raw/llm_aligned_0_1_huge_complex.json')